<a href="https://colab.research.google.com/github/farooqhasny/AGA8/blob/master/TF_TimeSerise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3MB 35.7MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23455 sha256=626a0b06b99d0b8bd6b980108cb12022b771dd5a11f1d702b122dda4ca566fcb
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
#- Stock market Price prediction - using past 60 days stock price
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
# import yfinance as yf
from google.colab import drive

root_path = 'gdrive/My Drive/your_project_folder/'  #change dir to your project folder

# New Section

In [ ]:
#Set Plot Style
plt.style.use('fivethirtyeight')
#Check Actions:
test = "STARTING TO ACQUIRE DATA FROM WEB"
print(test)
#Fetch Data from Server:
df = yf.download(tickers='AMZN', start='2021-06-01', end='2021-06-03', interval="1m")
# df = web.DataReader('CBA.AX', data_source='yahoo', start='2012-01-01', end='2019-12-17')
print(df)
compression_opts = dict(method='zip',archive_name='out.csv')
df.to_csv('out.zip', index=False,compression=compression_opts)

STARTING TO ACQUIRE DATA FROM WEB
[*********************100%***********************]  1 of 1 completed
                                  Open         High  ...    Adj Close  Volume
Datetime                                             ...                     
2021-06-01 09:30:00-04:00  3243.495117  3244.179932  ...  3244.179932   99620
2021-06-01 09:31:00-04:00  3241.399902  3244.000000  ...  3241.989990   11192
2021-06-01 09:32:00-04:00  3240.794922  3245.020020  ...  3239.899902   15200
2021-06-01 09:33:00-04:00  3239.000000  3242.989990  ...  3242.989990   13669
2021-06-01 09:34:00-04:00  3240.000000  3240.000000  ...  3239.886719    7047
...                                ...          ...  ...          ...     ...
2021-06-02 15:55:00-04:00  3230.729980  3231.159912  ...  3231.100098   10907
2021-06-02 15:56:00-04:00  3231.129883  3232.610107  ...  3232.159912   14835
2021-06-02 15:57:00-04:00  3231.840088  3232.360107  ...  3232.239990   15817
2021-06-02 15:58:00-04:00  3232.219971 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Get the number of rows and columns in the data set:
df.shape
print(df.shape)
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)')
#plt.show()

In [ ]:
#Create a new Dataframe - using only close columns:
data = df.filter(['Close'])
#Convert Dataframe to a numpy array
dataset = data.values
#Get the numbers of rows to train the model on:
training_data_len = math.ceil(len(dataset)* .8)
print(training_data_len)

#Scale the data (this is advantegeous to input data in p.u.)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
print(scaled_data)

#Create a training set of data:
train_data = scaled_data[0:training_data_len , :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<=61:
        print (x_train)
        print (y_train)
        print()
        
#Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
#reshape the data:
#x_train = np.reshape(x_train, (1543,60,1))
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

print(x_train.shape)
model = tf.keras.Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
test_data=scaled_data[training_data_len - 60:,:]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i,0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
#Get the models predicted price values:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#GEt the root mean squared root (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print (rmse)

In [ ]:
#Plot the Data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['predictions'] = predictions
# print(predictions)
#Visualize data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Predictions', fontsize=18)
plt.plot(train['Close'])

# plt.plot(valid[['Close', 'predictions']])
plt.plot(valid['predictions'])
plt.legend(['Train','Val','predictions'], loc='lower right')
plt.show()